<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, RandomizedSearchCV
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, StandardScaler, RobustScaler
import ast
from datetime import datetime
from time import gmtime, strptime, mktime
import ast
from dateutil.relativedelta import relativedelta
import re
import lightgbm as lgb
import json
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, StackingRegressor

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Setup

In [ ]:
VERSION    = 16
DIR_TRAIN  = '../input/parsing-auto-ru-24-01-2021/' # внешний датасет (парсинг сделан локально)
DIR_TRAIN_BASE  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # внешний датасет (парсинг сделан локально)
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [ ]:
!ls '../input'

In [ ]:
test = pd.read_csv(DIR_TEST+'test.csv')

In [ ]:
train_new = pd.read_csv(DIR_TRAIN+'auto_ru_24_01_2021.csv') # датасет для обучения модели

In [ ]:
train_old = pd.read_csv(DIR_TRAIN_BASE+'all_auto_ru_09_09_2020.csv') # датасет из бейзлайна

In [ ]:
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
# переименуем колонки для корректного объединения датасетов
new_columns_old = ['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'complectation_dict', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'start_date', 'hidden', 'model_name']
train_old.columns = new_columns_old

In [ ]:
# удалим несовпадающие колонки, которые не несут важной информации

train_old.drop(['hidden', 'start_date', 'Состояние'], axis = 1, inplace = True)
train_new.drop(['car_url','image',
 'model_info',
 'parsing_unixtime',
 'priceCurrency',
 'sell_id', 'Состояние'], axis = 1, inplace = True)
test.drop(['car_url','image',
 'model_info',
 'parsing_unixtime',
 'priceCurrency',
 'sell_id', 'Состояние'], axis = 1, inplace = True)

In [ ]:
# добавим в "старый" трейн значимые колонки
cols_to_add = ['equipment_dict', 'super_gen', 'vendor']
for col in cols_to_add:
    train_old.loc[:, col] = np.nan

In [ ]:
# добавим в тест колонку price, заполненную 0

test.loc[:, 'price'] = np.zeros(len(test))

In [ ]:
# заполним признак 'parsing_unixtime'
train_old.loc[:, 'parsing_unixtime'] = np.zeros(len(train_old))
train_new.loc[:, 'parsing_unixtime'] = np.ones(len(train_new))
train_new.loc[:, 'parsing_unixtime'] = train_new['parsing_unixtime']+1
test.loc[:, 'parsing_unixtime'] = np.ones(len(test))

## Data Preprocessing

In [ ]:
# сравним новый и старый трейны и тест датасеты, т.к. парсинг проводился в разное время (воспользуюсь функцией других участников)

def check_df_before_merg(d_df1,d_df2):
    
    list_of_names1 = list(d_df1.columns)
    temp_dict = {}
    temp_dict['feature_train'] = list_of_names1
    temp_dict['type_train'] = d_df1.dtypes
    temp_dict['sample_train'] = d_df1.loc[5].values
    temp_dict['# unique_train'] = d_df1.nunique().values
    temp_df1 = pd.DataFrame.from_dict(temp_dict)
    
    
    list_of_names2 = list(d_df2.columns)
    temp_dict2 = {}
    temp_dict2['feature_test'] = list_of_names2
    temp_dict2['type_test'] = d_df2.dtypes
    temp_dict2['sample_test'] = d_df2.loc[5].values
    temp_dict2['# unique_test'] = d_df2.nunique().values
    temp_df2 = pd.DataFrame.from_dict(temp_dict2)
    
    temp_insert = pd.DataFrame(columns=['< - >'])
    
    temp_df = pd.concat([temp_df1,temp_insert, temp_df2], axis=1, sort=False)
    temp_df.reset_index(inplace = True)
    del temp_df['index']
    temp_df['< - >'] = '| - |'
    display(temp_df)

    temp_dict3 = {}
    temp_df3= pd.DataFrame(temp_df)
    temp_list  = []
    temp_list2  = []
    temp_list3  = []
    temp_list4  = []
    temp_list5  = []

    for i in range(len(temp_df)):
        if str(temp_df3['type_train'][i]) != str(temp_df3['type_test'][i]):
            temp_list.append(temp_df3['feature_train'][i])
            temp_list2.append(temp_df3['feature_test'][i])
            temp_list3.append(str(temp_df3['type_train'][i]) + ' != ' + str(temp_df3['type_test'][i]))
            temp_list4.append(i)
        if temp_df3['# unique_test'][i]>0 and temp_df3['# unique_train'][i]/temp_df3['# unique_test'][i] > 2:
            temp_list5.append(i)
            
    temp_dict3['index']= temp_list4
    temp_dict3['feature_train']= temp_list
    temp_dict3['не совпадают типы'] = temp_list3
    temp_dict3['feature_test']= temp_list2

    temp_df4 = pd.DataFrame.from_dict(temp_dict3)
    temp_df4.set_index('index',inplace=True)

    print(f'Резюме:\n 1. Не совпали типы в:= {len(temp_df4)} столбцах\n')
    print(f'2. Уникальные значения различаются в:= {len(temp_list5)} столбцах {temp_list5}')
    display(temp_df4)

In [ ]:
check_df_before_merg(train_old,train_new)

In [ ]:
# приведем в соответствие трейны перед объединением
train_new.loc[:, 'engineDisplacement'] = train_new['engineDisplacement'].apply(lambda x: str(np.round(x/1000, 1)))

# обработаем признак 'vehicleConfiguration'
train_new.loc[:, 'vehicleConfiguration'] = train_new['vehicleConfiguration'].fillna(0).apply(lambda x: ast.literal_eval(x)['body_type'] if x!=0 else np.nan)
train_new[['vehicleConfiguration','vehicleTransmission','engineDisplacement']].fillna(0, inplace=True)

tmp = pd.Series(np.zeros(len(train_new)), name ='vehicleConfiguration')
for i in range(0, len(train_new)):
    if train_new['vehicleConfiguration'][i]!=0 and train_new['vehicleTransmission'][i]!=0 and train_new['engineDisplacement'][i]!=0:
        tmp[i] = f"{train_new['vehicleConfiguration'][i]} {train_new['vehicleTransmission'][i]} {train_new['engineDisplacement'][i]}"
train_new.loc[:, 'vehicleConfiguration'] = tmp

dict_fuel = {'GASOLINE':'бензин', 'DIESEL':'дизель', 'HYBRID':'гибрид', 
              'ELECTRO':'электро', 'LPG':'газ'}
train_new['fuelType'] = train_new['fuelType'].map(dict_fuel)

# обработаем признак Привод
train_new['Привод'] = train_new['Привод'].map({'FORWARD_CONTROL':'передний', 'ALL_WHEEL_DRIVE':'полный', 'REAR_DRIVE':'задний'})

In [ ]:
# исправим некорректные значения 'modelDate'

# index = []
# index = train_new[(train_new['productionDate'] - train_new['modelDate'])<0]['modelDate'].index.to_list()
# for x in index:
#     train_new.loc[x, 'modelDate'] =  train_new.loc[x, 'productionDate']

# index = train_old[(train_old['productionDate'] - train_old['modelDate'])<0]['modelDate'].index.to_list()
# for x in index:
#     train_old.loc[x, 'modelDate'] =  train_old.loc[x, 'productionDate']

# index = test[(test['productionDate'] - test['modelDate'])<0]['modelDate'].index.to_list()
# for x in index:
#     test.loc[x, 'modelDate'] =  test.loc[x, 'productionDate']

In [ ]:
# добавим признак modelDate_diff отдельно в тест и два трейна, т.к. у нас различается год
train_new.loc[:, 'modelDate_diff'] = (2021 - train_new['modelDate'])
train_old.loc[:, 'modelDate_diff'] = (2020 - train_old['modelDate'])
test.loc[:, 'modelDate_diff'] = (2020 - test['modelDate'])

# добавим признак productionDate_diff
train_new.loc[:, 'productionDate_diff'] = (2021 - train_new['productionDate'])
train_old.loc[:, 'productionDate_diff'] = (2020 - train_old['productionDate'])
test.loc[:, 'productionDate_diff'] = (2020 - test['productionDate'])

train_new.dropna(subset = ['modelDate_diff'], inplace=True)
train_old.dropna(subset = ['modelDate_diff'], inplace=True)

In [ ]:
train_new['modelDate_diff'] = train_new['modelDate_diff'].astype('int64')
train_old['modelDate_diff'] = train_old['modelDate_diff'].astype('int64')

In [ ]:
# объединим "старый" и "новый" трейны
train = train_new.append(train_old, sort=False).reset_index(drop=True) # объединяем

In [ ]:
# Введем признак разделения на тренировочную и тестовую выбороки - sample
train['sample'] = 1
test['sample'] = 0

In [ ]:
check_df_before_merg(train, test)

In [ ]:
# посмотрим на признак "bodyType"
train['bodyType'].value_counts()

In [ ]:
test['bodyType'].value_counts()

In [ ]:
# заменим заглавные буквы в названии на строчные
test['bodyType'] = test['bodyType'].apply(lambda x: str(x).lower().replace('.', ''))
train['bodyType'] = train['bodyType'].apply(lambda x: str(x).lower().replace('.', ''))

In [ ]:
# создадим список уникальных значений типа кузова из теста
body_type_list = list(test['bodyType'].unique())

In [ ]:
def get_body_type(x, body_type_list):
    for item in body_type_list:
        if str(item) in str(x):
            return item
        else: continue
    else: return 'иное'
train['bodyType'] = train['bodyType'].apply(lambda x: get_body_type(x, body_type_list))

In [ ]:
train['bodyType'].value_counts()

In [ ]:
# приведем обозначения цвета к одинаковым значениям

dict_color = {'040001':'чёрный', 'EE1D19':'красный', '0000CC':'синий', 
              'CACECB':'серебристый', '007F00':'зелёный', 'FAFBFB':'белый', 
              '97948F':'серый', '22A0F8':'голубой', '660099':'пурпурный', 
              '200204':'коричневый', 'C49648':'бежевый', 'DEA522':'золотистый', 
              '4A2197':'фиолетовый', 'FFD600':'жёлтый', 'FF8649':'оранжевый', 
              'FFC0CB':'розовый'}
train['color'] = train['color'].map(dict_color)

In [ ]:
# обработаем признак 'engineDisplacement'
test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: str(x).replace('LTR', ''))

In [ ]:
# обработаем признак enginePower
test['enginePower'] = test['enginePower'].apply(lambda x: str(x).replace('N12', '')).astype('float64')

In [ ]:
# обработаем признак 'fuelType'
test['fuelType'].value_counts()

In [ ]:
train['fuelType'].value_counts()

In [ ]:
dict_trans = {'AUTOMATIC':'автоматическая', 'MECHANICAL':'механическая', 'ROBOT':'роботизированная', 
              'VARIATOR':'вариатор'}
train['vehicleTransmission'] = train['vehicleTransmission'].map(dict_trans)

In [ ]:
# modelDate, numberofDoors - заполним пропуски и приведем формат в соответствие тестовому датасету
for col in ['modelDate', 'numberOfDoors']:
    train[col] = train[col].fillna(0).astype('int')

In [ ]:
# посмотрим на столбец "Владельцы"
test['Владельцы'].value_counts()

In [ ]:
# приведем в соответствие тесту
train['Владельцы'] = train['Владельцы'].map({4.0:'3 или более', 3.0:'3 или более', 2.0:'2 владельца', 1.0:'1 владелец'})

In [ ]:
# признак Владение не заполнен в более чем 65% случаев, возможно этот признак заполняют реальные владельцы автомобилей, но не посредники, например, создадим бинарный признак 
test['Владение'] = test['Владение'].fillna(0)
test['Владение'] = test['Владение'].apply(lambda x: 1 if x!=0 else 0)
train['Владение'] = train['Владение'].fillna(0)
train['Владение'] = train['Владение'].apply(lambda x: 1 if x!=0 else 0)

In [ ]:
# train['Владение'] = train['Владение'].fillna(0).apply(lambda x: datetime.fromtimestamp(mktime(strptime(x, "{'year': %Y, 'month': %m}"))) if x!=0 else x)

# def time_fix(x):
#     _years = 'лет'
#     _months = 'месяцев'
#     if relativedelta(datetime.now(),x).years in [1, 21, 31, 41]:
#         _years = 'год'
#     if relativedelta(datetime.now(),x).years in [2, 3, 4, 21, 22, 23, 24, 31, 32, 33, 34, 41, 42, 43, 44]:
#         _years = 'года'
    
#     if relativedelta(datetime.now(),x).months==1:
#         _months = 'месяц'
#     if relativedelta(datetime.now(),x).months in [2, 3, 4]:
#         _months = 'месяца'
   
#     if relativedelta(datetime.now(),x).years!=0 and relativedelta(datetime.now(),x).months!=0:
#         return f'{relativedelta(datetime.now(),x).years} {_years} и {relativedelta(datetime.now(),x).months} {_months}'
#     if relativedelta(datetime.now(),x).years==0 and relativedelta(datetime.now(),x).months!=0:
#         return f'{relativedelta(datetime.now(),x).months} {_months}'
#     if relativedelta(datetime.now(),x).years!=0 and relativedelta(datetime.now(),x).months==0:
#         return f'{relativedelta(datetime.now(),x).years} {_years}'
#     else:
#         return 0
    
# train['Владение'] = train['Владение'].apply(lambda x: time_fix(x))

In [ ]:
# обработаем признак ПТС
train['ПТС'] = train['ПТС'].map({'ORIGINAL':'Оригинал', 'DUPLICATE':'Дубликат'})

In [ ]:
# обработаем признак Руль
train['Руль'] = train['Руль'].map({'RIGHT':'Правый', 'LEFT':'Левый'})

In [ ]:
# обработаем признак Таможня
train['Таможня'] = train['Таможня'].map({True: 'Растаможен', False:'Не растаможен'})

In [ ]:
check_df_before_merg(train, test)

In [ ]:
# убедимся что в столбце 'price' трейна нет пропущенных значений
train[train['price'].isnull()]

In [ ]:
# удалим строки с пропущенными ценами
train.dropna(subset=['price'], inplace=True)

In [ ]:
# сгруппируем марки, которые не встречаются в тесте
brand_list_test = list(test['brand'].unique())
brand_list_test

In [ ]:
train['brand'] = train['brand'].apply(lambda x: x.upper())
train['brand'] = train['brand'].apply(lambda x: x.replace('MERCEDES-BENZ', 'MERCEDES'))

In [ ]:
brand_list_train = list(train['brand'].unique())
brand_list_train

In [ ]:
train['brand'].replace([x for x in brand_list_train if x not in brand_list_test], np.nan, inplace=True)

In [ ]:
# заменим марки, которые не встречаются в тесте, на OTHER
train['brand'].fillna('OTHER', inplace=True)

In [ ]:
train['brand'].value_counts()

In [ ]:
# отброс данных по маркам, которых нет в тесте не улучшил скор, поэтому оставим для лучшей генерализации модели
# train.dropna(subset=['brand'], inplace = True)

In [ ]:
# удаляем дубликаты строк в трейне
train.drop_duplicates(keep='first', inplace=True, ignore_index=True)

In [ ]:
# для корректной обработки признаков объединяем трейн и тест в один датасет
data = test.append(train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
# добавим годовой пробег
data.loc[:, 'mileage_per_annum'] = data['mileage']//(data['productionDate_diff']+1)

In [ ]:
# посмотрим на признаки, которые содержат словари, позже можно будет попробовать сгенерить на их основе доп. features
data['complectation_dict'].sample(15)

In [ ]:
# у этого признака очень много пропущенных значений, использовать не получится
data['complectation_dict'].isnull().sum()

In [ ]:
# обработаем признак description
data['description'].sample()

In [ ]:
# попробуем по ключевым словам отсортировать объявления от компаний и частных лиц (для этого я воспозуюсь функцией, которую создавала для ресторанного рейтинга TripAdvisor)

words_owner = ["беспокоить", "битый", "битая", "не нужд", "хозяин"]
words_dealer = ['автомобилей с пробегом', "предпродажн", "trade", "тест-драйв", "клиент", "в налич", "онлайн", "трейд", "юр", "предлож", "наличии", "программ", "выкуп"]

import re

def review_analysis_column(x,words_owner, words_dealer):
    result_owner = 0
    result_dealer = 0
    for word in words_owner:
        if word in x:
            result_owner+=1
    for word in words_dealer:
        if word in x:
            result_dealer+=1
    if result_dealer>result_owner:
        return 'Дилер'
    else:
        return 'Владелец'

data.loc[:, 'owner_type'] = data['description'].apply(lambda x: review_analysis_column(str(x), words_owner, words_dealer))

In [ ]:
data['owner_type'].value_counts()

In [ ]:
# посмотрим словарь 'equipment_dict'
data['equipment_dict'].sample(100)

In [ ]:
# создадим список опций
options_list = ['cruise-control', 'leather', 'esp','adaptive-light','usb','sport-seats','multi-wheel','xenon','ashtray-and-cigarette-lighter','airbag-passenger','front-centre-armrest','navigation', 'lock',
                'rear-camera','door-sill-panel','servo','steering-wheel-gear-shift-paddles','electro-mirrors','activ-suspension','electro-window-back','reduce-spare-wheel','mirrors-heat',
                'park-assist-f','seat-memory','leather','19-inch-wheels','wheel-heat','led-lights','music-super','park-assist-r','body-kit','start-stop-function','airbag-driver','aux'
                ,'isofix','electro-window-front','light-sensor','hcc','automatic-lighting-control','airbag-curtain','passenger-seat-updown','high-beam-assist','computer','keyless-entry',
                'seat-transformation','passenger-seat-electric','alarm','light-cleaner','paint-metallic','ptf','start-button','rain-sensor','airbag-side','tyre-pressure','electro-trunk',
                'abs','bluetooth','front-seats-heat','wheel-leather','wheel-configuration2','wheel-configuration1','immo','climate-control-1','12v-socket','third-rear-headrest']    

In [ ]:
# добавим признак количество опций
def options(x, options_list):
    num=0
    for option in options_list:
        if option in x:
            num+=1
    return num
                
data.loc[:, 'options_number'] = data['equipment_dict'].fillna(0).apply(lambda x: options(x, options_list) if x!=0 else 0)

# признак работал гораздо лучше без исполозования трейна из бейзлайна, но так как спарсить исторические данные не представляется возможным, а с момента парсинга
# тестового датасета прошло больше 3-х месяцев + переход через год, поэтому цены изменились

In [ ]:
data['options_number'].value_counts()

In [ ]:
# посмотрим словарь 'super_gen', увидим в нем интересный признак 'price_segment'

data['super_gen'].sample(10)

In [ ]:
data['super_gen'].fillna(0, inplace=True)

In [ ]:
data['super_gen'] = data['super_gen'].apply(lambda x: str(x).replace('"',"'") if x!=0 else 0)

In [ ]:
# добавим признак 'price_segment'
data['price_segment'] = data['super_gen'].apply(lambda x: ast.literal_eval(x) if x!=0 else np.nan)

In [ ]:
m = type(data['price_segment'][67725])

In [ ]:
def price_segment(x, m):
    if type(x)== m:
        if 'price_segment' in x.keys():
            return x['price_segment']
    else:
        return 'OTHER'

data['price_segment'] = data['price_segment'].apply(lambda x: price_segment(x, m))

In [ ]:
data['price_segment'].value_counts()

In [ ]:
data['price_segment'].fillna('OTHER', inplace=True)

In [ ]:
# заполним признак 'engineDisplacement' на основе признака 'name'
pattern = re.compile('(\d{1}\.\d{1})')
data['engineDisplacement'] = data['name'].apply(lambda x:str(pattern.findall(str(x)))[2:5])
data['engineDisplacement'] = pd.to_numeric(data['engineDisplacement'], errors='coerce')

In [ ]:
# удалим столбцы со словарями  
to_drop = ['complectation_dict', 'description', 'equipment_dict', 'super_gen','name', 'Таможня']
data.drop(columns=to_drop, inplace=True)

In [ ]:
# заполним столбцы с пропусками
data.columns[data.isna().any()].tolist()

In [ ]:
data[data['engineDisplacement'].isnull()]

In [ ]:
# видно, что значения engineDisplacement отсутствуют для электрокаров, заполним их нулями
data['engineDisplacement'].fillna(0, inplace=True)

In [ ]:
data[data['Владельцы'].isnull()]

In [ ]:
# в столбце Владельцы информация отсутствует для новых машин без пробега, заполним 0
data['Владельцы'].fillna('0 владельцев', inplace=True)

In [ ]:
data[data['ПТС'].isnull()]

In [ ]:
# данные о ПТС также отсутствуют в основном для новых машин, заполним их значением "Оригинал"
data['ПТС'].fillna("Оригинал", inplace=True)

In [ ]:
# заполним цены в тесте нулями
data['price'].fillna(0, inplace=True)

In [ ]:
data['vendor'].fillna("OTHER", inplace=True)

In [ ]:
data['parsing_unixtime'] = data['parsing_unixtime'].astype('int64')

In [ ]:
# убедимся, что пропущенные значения отсутствуют
data.columns[data.isna().any()]

In [ ]:
data.info()

## Корреляция и сортировка признаков

In [ ]:
data.columns

In [ ]:
# переименуем столбцы
new_columns = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower',
       'fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors',
       'productionDate', 'vehicleConfiguration', 'vehicleTransmission',
       'vendor', 'owners', 'ownership', 'certificate', 'drive', 'wheel', 'price',
       'parsing_unixtime', 'modelDate_diff', 'productionDate_diff', 'sample',
       'mileage_per_annum', 'owner_type', 'options_number', 'price_segment']
data.columns = new_columns

In [ ]:
# выделим категориальные признаки
cat_cols = data.select_dtypes(include=object).columns.to_list()

In [ ]:
# выделим числовые и бинарные и ранговые признаки
num_cols = list(set(new_columns) - set(cat_cols))
num_cols.remove('price')
num_cols.remove('sample')

In [ ]:
bin_rank_cols = ['parsing_unixtime', 'numberOfDoors', 'ownership']
num_cols = list( set(num_cols) - set(bin_rank_cols))

In [ ]:
# посмотрим распределение числовых признаков
for i in num_cols:
    plt.figure()
    sns.distplot(data[i].dropna(), kde = False, rug=False, color='b')
    plt.title(i)
    plt.show()

# логарифмирование числовых переменных скор не улучшило

In [ ]:
# построим heat-map числовых (непрерывных) переменных
fig, ax = plt.subplots(figsize=(10,5))
sns.heatmap(data[num_cols].corr().abs(), vmin=0, vmax=1, annot=True, ax=ax)

In [ ]:
# увидим, что есть корреляция между признаками, но так как мы собираемся строить модели с решающими деревьями, не факт, что она повлияет на скор. Признаки удалять пока
# не будем, проверим сначала влияние на скор

In [ ]:
# посмотрим корреляцию Спирмана для ранговых и бинарных переменных
data[bin_rank_cols+['price']].corr(method='spearman')

In [ ]:
data[num_cols].describe()

In [ ]:
# посмотрим на значимость числовых признаков для модели
data_temp = data[data['sample']==1]
imp_num = pd.Series(f_classif(data_temp[num_cols], data_temp['price'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
plt.figure(figsize=(10,5))
imp_num.plot(kind = 'barh')

In [ ]:
# видим большую зависимость цены от даты производства и года выхода модели, что потом подтвердит feature importance моделей

In [ ]:
# закодируем категориальные признаки при помощи LabelEncoding (OneHotEncoding дает хуже результат по скору)
label_encoder = LabelEncoder()
for column in cat_cols:
    data[column] = label_encoder.fit_transform(data[column])

In [ ]:
# посмотрим на важность бинарных и категориальных признаков для модели

data_temp = data[data['sample']==1]

mp_cat = pd.Series(mutual_info_classif(data_temp[cat_cols+bin_rank_cols], data_temp['price'],
                                    discrete_features =True), index = cat_cols+bin_rank_cols)
mp_cat.sort_values(inplace = True)
plt.figure(figsize=(10,5))
mp_cat.plot(kind = 'barh')

In [ ]:
# видим большую зависимость цены от названия модели и признака vehicleConfiguration, который мы сгенерили

In [ ]:
X =data.query('sample == 1').drop(['sample', 'price'], axis=1)
y = data.query('sample == 1')['price']
X_sub = data.query('sample == 0').drop(['sample', 'price'], axis=1)

## Train Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

## Data Scaling

In [ ]:
# применение MinMaxScaler к тесту и трейну не улучшило скор
# x_scaler = MinMaxScaler()
# x_scaler = x_scaler.fit(X_train)
# x_scaler.transform(X_train)
# x_scaler.transform(X_test)
# x_scaler.transform(X_sub)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())
#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

# # Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
#          cat_features=cat_cols,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [ ]:
# оцениваем точность
predict = model.predict(X_test)

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True)
                         
model.fit(X_train, np.log(y_train),
#          cat_features=cat_cols,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

In [ ]:
predict_test = np.exp(model.predict(X_test))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# GridSearch : CatBoost

In [ ]:
# запустим простой поиск параметров

model = CatBoostRegressor(random_seed = RANDOM_SEED, eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],   loss_function = 'MAE')

grid = {'learning_rate': [0.01, 0.03, 0.05, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
        'leaf_estimation_iterations': [1, 5, 10]       
       }

randomized_search_result = model.randomized_search(grid,
                                                   X=X,
                                                   y=np.log(y), cv=5,
                                                   plot=True)

In [ ]:
randomized_search_result['params']

In [ ]:
# построим модель с лучшими найденными параметрами
best_cat_model = CatBoostRegressor(leaf_estimation_iterations = 1,
 depth = 10,
 l2_leaf_reg = 3,
 learning_rate = 0.05,
 random_seed = RANDOM_SEED,
 eval_metric = 'MAPE',
 custom_metric = ['R2', 'MAE'],
 silent = True,
 iterations = 5000)
                         
best_cat_model.fit(X_train, np.log(y_train),
#          cat_features=cat_cols+bin_rank_cols,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

In [ ]:
best_cat_model_predict = np.exp(best_cat_model.predict(X_test))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, best_cat_model_predict))*100:0.2f}%")

In [ ]:
# даже такой простой подбор параметров позволил снизить ошибку с 11.73% до 11.08%

# Model 3 : LightGBM

In [ ]:
# попробуем запустить модель градиентного бустинга LightGBM
lgb_train = lgb.Dataset(X_train, np.log(y_train))

In [ ]:
lgb_eval = lgb.Dataset(X_test, np.log(y_test), reference=lgb_train)

In [ ]:
# params = {
#         'task': 'train',
#         'boosting_type': 'gbdt',
#         'objective': 'regression',
#         'boost_from_average': False,        
#         'metric': {'MAPE'},
#         'nthread': -1,
#         'random_state': RANDOM_SEED,
#         'verbose': 0
# }

params_optuna = {'max_depth': 12, 'learning_rate': 0.09886252405243971, 'feature_fraction': 0.9730539377362546, 'bagging_fraction': 0.9880829822536809, 
      'bagging_freq': 3, 'drop_rate': 0.009422844803015881, 'skip_drop': 0.17709716314580068, 'metric': {'MAPE'}, 'boost_from_average': False, 'random_state': RANDOM_SEED}

In [ ]:
gbm = lgb.train(params_optuna, lgb_train, num_boost_round=1000, valid_sets=lgb_eval, early_stopping_rounds=50)

In [ ]:
# построим график feature importance
lgb.plot_importance(gbm, importance_type='gain', figsize=(20,10))

In [ ]:
pred_gbm = np.exp(gbm.predict(X_test))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, pred_gbm))*100:0.2f}%")

In [ ]:
# попробуем запустить optuna для подбора параметров

# попробуем запустить модель градиентного бустинга LightGBM
lgb_train = lgb.Dataset(X_train, np.log(y_train))

import sklearn
import optuna

def objective(trial):       

    params = {"objective": "regression",
        "boost_from_average": False,
        "seed": RANDOM_SEED,
        "verbosity": -30,
        "num_threads": -1,
        "max_depth": trial.suggest_int('max_depth', 8, 12),
#         "num_leaves": 2**'max_depth',
        "num_iterations": 1000,
        "learning_rate": trial.suggest_loguniform('learning_rate', 1e-2, 0.1),
        "feature_fraction": trial.suggest_uniform('feature_fraction', 0.3, 1.0),
        "bagging_fraction": trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
        "bagging_freq": trial.suggest_int('bagging_freq', 1, 4),
        "drop_rate": trial.suggest_loguniform("drop_rate", 1e-8, 1.0),
        "skip_drop": trial.suggest_loguniform("skip_drop", 1e-8, 1.0),
        'metric': {'MAPE'},
#         "max_bin": 2**trial.suggest_int('max_bin', 1, 2,  8)-1,
#         "min_child_samples": 2**trial.suggest_int('min_child_samples', 1, 2, 4, 12),
             }   


    gbm = lgb.train(params,
                lgb_train)   

    y_pred = np.exp(gbm.predict(X_test))
    return mape(y_test, y_pred)


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)


print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
# результат ниже чем у модели Catboost

# Model 4 : Random Forest

In [ ]:
# попробуем построить модель Random Forest c подбором параметров

# random_grid = {'n_estimators': [100, 200, 300],
#                'max_features': ['auto', 'sqrt'],
#                'max_depth': [None],
#                'min_samples_split': [2, 5, 10],
#                'min_samples_leaf': [1, 2, 4],
#                'bootstrap': [True, False]}

# rfr = RandomForestRegressor(random_state = RANDOM_SEED)
# rf_random = RandomizedSearchCV(estimator = rfr, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=10, random_state=RANDOM_SEED, n_jobs = -1)
# rf_random.fit(X_train, np.log(y_train))
# rf_random.best_params_

In [ ]:
# rfr = RandomForestRegressor(best_params, random_state = RANDOM_SEED)
best_rfr = RandomForestRegressor(random_state=RANDOM_SEED
                      , n_estimators=300
                      , min_samples_split=5
                      , min_samples_leaf=2
                      , max_features='sqrt'
                      , max_depth=None
                      , bootstrap=False)

best_rfr.fit(X_train, np.log(y_train))


predict_rfr = np.exp(best_rfr.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_rfr))*100:0.2f}%")

# Model 5 : Бэггинг

In [ ]:
# попробуем бэггинг со случайным лесом

bag_rfr = BaggingRegressor(best_rfr, n_estimators=3, n_jobs=1, random_state=RANDOM_SEED)
bag_rfr.fit(X_train, np.log(y_train))
predict_bag_rfr = np.exp(bag_rfr.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_bag_rfr))*100:0.2f}%")

# Model 5 : Стэкинг

In [ ]:
# попробуем сделать стэкинг моделей
estimators=[('b_gbr', BaggingRegressor(best_rfr
                                        ,n_estimators=3
                                        ,n_jobs=1
                                        ,random_state=RANDOM_SEED))
            ,('LightGBM', lgb.LGBMRegressor(
                                    objective='regression',
                                    n_estimators=100, 
                                    random_state=RANDOM_SEED))]

st_ensemble = StackingRegressor(estimators=estimators
                                ,final_estimator = CatBoostRegressor(leaf_estimation_iterations = 1,
                                                                     depth = 10,
                                                                     l2_leaf_reg = 3,
                                                                     learning_rate = 0.05,
                                                                     random_seed = RANDOM_SEED,
                                                                     eval_metric = 'MAPE',
                                                                     custom_metric = ['R2', 'MAE'],
                                                                     silent = True,
                                                                     iterations = 5000))
                                        
st_ensemble.fit(X_train, np.log(y_train))

predict_ensemble = np.exp(st_ensemble.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_ensemble))*100:0.2f}%")

In [ ]:
# сделаем submission по лучшей модели
predict_submission = np.exp(best_cat_model.predict(X_sub))

# Submission

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)

### Значение MAPE на лидерборде 11,89%

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing